In [1]:
import configparser
import os

In [2]:
# get user inputs from config file:
config_file = os.path.abspath("config.ini")
config = configparser.ConfigParser()
config.read(config_file)
input_dict = config['input']
process_dict = config['process']
output_dict = config['output']

In [3]:
##################################
# Read your input data from config file
##################################
aoi = os.path.join('input/', input_dict['aoi'])
## Accessibility
road = os.path.join('input/', input_dict['road'])
settlement = os.path.join('input/', input_dict['settlement'])
river = os.path.join('input/', input_dict['river'])

## Topography
elevation = os.path.join('input/', input_dict['elevation'])
slope = os.path.join('input/', input_dict['slope'])

## Forest or deforestation history
forest_cover_t1 = os.path.join('input/', input_dict['forest_cover_t1'])
forest_cover_t2 = os.path.join('input/', input_dict['forest_cover_t2'])
forest_cover_t3 = os.path.join('input/', input_dict['forest_cover_t3'])

## Land conservation status
protected_area = os.path.join('input/', input_dict['protected_area'])

In [4]:
# reload all modules automatically when changes happen
%load_ext autoreload
%autoreload 2

In [9]:
##################################
# Processing data. Read from process config file
##################################

road_process = os.path.join('process/', process_dict['road'])
settlement_process = os.path.join('process/', process_dict['settlement'])
forest_cover_change_t12 = os.path.join('process/', process_dict['forest_cover_change_t12'])
forest_cover_change_t23 = os.path.join('process/', process_dict['forest_cover_change_t23'])

distance_deforestation_t12 = os.path.join('process/', process_dict['distance_deforestation_t12'])
distance_deforestation_t23 = os.path.join('process/', process_dict['distance_deforestation_t23'])

distance_road = os.path.join('process/', process_dict['distance_road'])
distance_river = os.path.join('process/', process_dict['distance_river'])

In [5]:
from models import ProcessingForecastDeforestation

process = ProcessingForecastDeforestation(target_crs=32749)


==================== BEGINNING LOG ====================


In [10]:
##################################
# Run the process. Initialize the input model
##################################

%time
process.clip_raster_to_aoi(settlement, aoi, settlement_process)
process.rasterize(road, road_process)
process.compute_deforestation(forest_cover_t1, forest_cover_t2, forest_cover_change_t12, nodata=None)
process.compute_deforestation(forest_cover_t2, forest_cover_t3, forest_cover_change_t23, nodata=None)

CPU times: total: 0 ns
Wall time: 0 ns


Clipped raster written: process/settlement.tif
Process took -1752134395.05 seconds
Raster written: process/road.tif (12154x3685, pixel size: 30 units)
Process took -1752134390.27 seconds
Deforestation raster saved to process/forest_cover_change_t12.tif
Process took -1752134406.40 seconds
Deforestation raster saved to process/forest_cover_change_t23.tif
Process took -1752134413.62 seconds


'process/forest_cover_change_t23.tif'

In [11]:
### Compute the Euclidean Distance as a means of estimating the probability of deforestation

#### Read the paper refence here: https://joss.theoj.org/papers/10.21105/joss.02975

In [ ]:
%time

process.euclidean_distance(road_process, distance_road, 100)
process.euclidean_distance(forest_cover_change_t12, distance_deforestation_t12, 100)
process.euclidean_distance(forest_cover_change_t23, distance_deforestation_t23", 100)